In [ ]:
import sys
import os
import cv2
import numpy as np
from tqdm import tqdm 
import json
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.append(os.path.dirname(os.getcwd()))
from pkg.teeth_detector import detect_teeth

In [ ]:
DATA_DIR = "../experiments/data/teeth_detector"      
RESULTS_DIR = "../experiments/results/teeth_detector"
UPSAMPLE_SCALE = 2.5
DRAW_CONFIDENCE = False
CONFIDENCE_THRESHOLD = 0.8  

In [ ]:
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
image_files = [
    f for f in os.listdir(DATA_DIR) 
    if any(f.lower().endswith(ext) for ext in image_extensions)
]

In [ ]:
print(f"Найдено изображений для обработки: {len(image_files)}")
if not image_files:
    raise ValueError(f"В папке {DATA_DIR} не найдено изображений")

In [ ]:
all_confidences = []

for img_file in tqdm(image_files, desc="Обработка изображений"):
    img_path = os.path.join(DATA_DIR, img_file)
    base_name = Path(img_file).stem
    
    image = cv2.imread(img_path)
    if image is None:
        print(f"Предупреждение: не удалось загрузить {img_file}")
        continue
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    orig_h, orig_w = image_rgb.shape[:2]
    
    new_w, new_h = int(orig_w * UPSAMPLE_SCALE), int(orig_h * UPSAMPLE_SCALE)
    vis_image = cv2.resize(image_rgb, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    scale_x, scale_y = new_w / orig_w, new_h / orig_h
    
    bboxes = detect_teeth(image_rgb)
    confidences = [bbox[4] for bbox in bboxes]
    all_confidences.extend(confidences)
    
    filtered_bboxes = [bbox for bbox in bboxes if bbox[4] >= CONFIDENCE_THRESHOLD]
    
    scaled_bboxes = []
    for bbox in filtered_bboxes:
        x1, y1, x2, y2, conf = bbox
        x1_s = int(x1 * scale_x)
        y1_s = int(y1 * scale_y)
        x2_s = int(x2 * scale_x)
        y2_s = int(y2 * scale_y)
        scaled_bboxes.append((x1_s, y1_s, x2_s, y2_s, conf))
    
    print(f"\n{img_file}: обнаружено {len(bboxes)} зубов")
    print(f"Значения уверенности: {[round(conf, 4) for conf in confidences]}")
    print(f"Средняя уверенность: {np.mean(confidences):.4f}")
    
    stats = {
        "original_image": img_file,
        "total_teeth_detected": len(bboxes),
        "teeth_after_threshold": len(filtered_bboxes),
        "confidence_threshold": CONFIDENCE_THRESHOLD,
        "confidences": confidences,
        "average_confidence": float(np.mean(confidences)) if confidences else 0.0
    }
    
    json_path = os.path.join(RESULTS_DIR, f"{base_name}.json")
    with open(json_path, 'w') as f:
        json.dump(stats, f, indent=4)
    
    for i, (x1, y1, x2, y2, conf) in enumerate(scaled_bboxes):
        cv2.rectangle(vis_image, (x1, y1), (x2, y2), (0, 255, 0), 3)
        
        if DRAW_CONFIDENCE:
            label = f"{conf:.2f}"
            cv2.putText(vis_image, label, (x1, y1 - 5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 3)
    
    output_path = os.path.join(RESULTS_DIR, f"{base_name}.jpg")
    cv2.imwrite(output_path, cv2.cvtColor(vis_image, cv2.COLOR_RGB2BGR))

In [ ]:
if all_confidences:
    print("\n" + "="*50)
    print("ОБЩАЯ СТАТИСТИКА ПО ВСЕМ ИЗОБРАЖЕНИЯМ")
    print(f"Всего обнаружено зубов: {len(all_confidences)}")
    print(f"Средняя уверенность: {np.mean(all_confidences):.4f}")
    print(f"Минимальная уверенность: {min(all_confidences):.4f}")
    print(f"Максимальная уверенность: {max(all_confidences):.4f}")
    print("="*50)
else:
    print("\nНе обнаружено ни одного зуба во всех изображениях.")